In [10]:
from openai import OpenAI
TOGETHER_AI_API_KEY = 'f41b1428204c49e0c8ae6f7b020557297806c591b1800d7cb2d241ab5b065805'
def generate_diffuser_text(thread_info: str, toxic_comments: list) -> str:
    """
    Generate a response aimed at diffusing the toxicity identified in the thread,
    incorporating the context of the thread and the toxic comments.

    :param thread_info: A string containing the compiled content of the thread.
    :param toxic_comments: A list of strings, each a comment identified as toxic.
    :return: A string containing the AI-generated diffuser text.
    """
    client = OpenAI(api_key=TOGETHER_AI_API_KEY = 'f41b1428204c49e0c8ae6f7b020557297806c591b1800d7cb2d241ab5b065805'
, base_url="https://api.together.xyz")
    toxic_summary = " ".join(toxic_comments)  # Combine toxic comments into a single string for simplicity
    diffuser_prompt = f"With the context of this thread: {thread_info} and the following toxic comments: {toxic_summary}, generate a response that can help diffuse the situation."
    chat_completion = client.chat.completions.create(
        messages=[
            {"role": "system", "content": "You are an AI trained to promote positive interactions and mitigate toxicity. Here's the context and toxic comments you need to address."},
            {"role": "user", "content": diffuser_prompt}
        ],
        model="google/gemma-7b-it",
    )
    diffuser_text = chat_completion.choices[0].message.content
    return diffuser_text

In [3]:
import os

from atproto import (
    CAR,
    AtUri,
    Client,
    FirehoseSubscribeReposClient,
    firehose_models,
    models,
    parse_subscribe_repos_message,
)
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Bluesky credentials
BLUESKY_USERNAME = "paritoshk.bsky.social"
BLUESKY_PASSWORD = "Pari2798!"

# Create a Bluesky client
client = Client("https://bsky.social")
firehose = FirehoseSubscribeReposClient()

In [7]:
def process_operation(
    op: models.ComAtprotoSyncSubscribeRepos.RepoOp,
    car: CAR,
    commit: models.ComAtprotoSyncSubscribeRepos.Commit,
):
    uri = AtUri.from_str(f"at://{commit.repo}/{op.path}")

    if op.action == "create":
        if not op.cid:
            return

        record = car.blocks.get(op.cid)
        if not record:
            return

        record = {
            "uri": str(uri),
            "cid": str(op.cid),
            "author": commit.repo,
            **record,
        }
    

        if uri.collection == models.ids.AppBskyFeedPost:
            # This logs the text of every post off the firehose.
            # Just for fun :)
            # Delete before actually using
            print(record['text'])
        
            if "stop-tox" in record["text"]:
                # get some info about the poster, their posts, and the thread they tagged the bot in
                poster_posts = client.get_author_feed(
                    actor=record["author"], cursor=None, filter=None, limit=100
                ).feed
                poster_follows = client.get_follows(actor=record["author"]).follows
                poster_profile = client.get_profile(actor=record["author"])
                posts_in_thread = client.get_post_thread(uri=record["uri"])

                # send a reply to the post
                record_ref = {"uri": record["uri"], "cid": record["cid"]}
                reply_ref = models.AppBskyFeedPost.ReplyRef(
                    parent=record_ref, root=record_ref
                )
                client.send_post(
                    reply_to=reply_ref,
                    text=f"Hey, {poster_profile.display_name}. You have {len(poster_posts)} posts and {len(poster_follows)} follows. Your bio is: {poster_profile.description}. There are {len(posts_in_thread)} posts in the thread.",
                )

        # elif uri.collection == models.ids.AppBskyFeedLike:
        #     print("Created like: ", record)
        # elif uri.collection == models.ids.AppBskyFeedRepost:
        #     print("Created repost: ", record)
        # elif uri.collection == models.ids.AppBskyGraphFollow:
        #     print("Created follow: ", record)

    if op.action == "delete":
        # Process delete(s)
        return

    if op.action == "update":
        # Process update(s)
        return

    return posts_in_thread

In [6]:
def on_message_handler(message: firehose_models.MessageFrame) -> None:
    commit = parse_subscribe_repos_message(message)
    if not isinstance(
        commit, models.ComAtprotoSyncSubscribeRepos.Commit
    ) or not isinstance(commit.blocks, bytes):
        return
    car = CAR.from_bytes(commit.blocks)
    for op in commit.ops:
        process_operation(op, car, commit)


def main() -> None:
    client.login(BLUESKY_USERNAME, BLUESKY_PASSWORD)
    print("🤖 Bot is listening")
    firehose.start(on_message_handler)


In [8]:
client = OpenAI(api_key=TOGETHER_AI_API_KEY, base_url="https://api.together.xyz")

In [11]:
chat_completion = client.chat.completions.create(
        messages=[
            {"role": "system", "content": "You are an AI trained to mitigate toxic conversations and promote positive interactions. Here's the context and what you need to address: " + "Nina: Fuck you, Tosh: Sorry please, Nina: Nice goodbye"},
            {"role": "user", "content": "stoptox now"}
        ],
        model="google/gemma-7b-it",  # Make sure to replace with the correct model identifier if needed
    )


InternalServerError: Error code: 500 - {'error': {'message': '(unknown path)\n  Error: Unable to call `raise_exception`, which is undefined or falsey', 'type': 'server_error', 'param': None, 'code': None}}

In [17]:
# Mock function to simulate replicate.run behavior
import replicate 
REPLICATE_API_TOKEN = 'r8_7ICSWEeEQdH6SyLPpcjU1723lKbVExK1kqyIx'
def send_prompt_to_ai(posts_in_thread, toxic_message):
    """
    Generates a non-toxic, diffusing response for a given toxic message within a thread context.

    :param posts_in_thread: List of messages in the thread leading up to the toxic message.
    :param toxic_message: The message flagged as toxic.
    :return: A string containing the AI-generated response to mitigate the conversation's toxicity.
    """

    # Prepare the thread context and the toxic message as a prompt for the AI model
    thread_context = " ".join([post['text'] for post in posts_in_thread])
    prompt = f"Given the following conversation thread: {thread_context} And a toxic message: {toxic_message} Generate a constructive and non-toxic response to diffuse the situation."

    output = replicate.run(
        "google-deepmind/gemma-7b-it:2790a695e5dcae15506138cc4718d1106d0d475e6dca4b1d43f42414647993d5",
        input={
            "top_k": 50,
            "top_p": 0.95,
            "prompt": prompt,
            "temperature": 0.7,
            "max_new_tokens": 512,
            "min_new_tokens": -1,
            "repetition_penalty": 1
        }
    )

    # Collect and return the response
    response = ""
    for item in output:
        response += item["text"]
    
    return response


# Example posts in the thread
posts_in_thread = [
    {"text": "I really enjoyed the event, it was very well organized."},
    {"text": "Absolutely, kudos to everyone involved."},
    {"text": "I didn't like it. Felt it was a waste of time."},
]

# Example toxic message
toxic_message = "I didn't like it. Felt it was a waste of time."



In [20]:
send_prompt_to_ai(posts_in_thread,toxic_message)

"It's important to remember the impact our words can have. Let's strive for understanding and kindness."

In [21]:
# Assuming the structure of the thread is provided in a variable named `thread` as described in your input.
# Also assuming the `replicate` library is properly imported and configured.
thread = "thread=ThreadViewPost(post=PostView(author=ProfileViewBasic(did='did:plc:ygxp5kv5dzeaglhbysa7a5ph', handle='paritoshk.bsky.social', avatar='https://cdn.bsky.app/img/avatar/plain/did:plc:ygxp5kv5dzeaglhbysa7a5ph/bafkreifpgid6nusmgdrj7kbqyeu4admxtu42njoei5wvxz4oda36ty5lne@jpeg', display_name='Paritosh K', labels=[], viewer=ViewerState(blocked_by=False, blocking=None, blocking_by_list=None, followed_by=None, following=None, muted=False, muted_by_list=None, py_type='app.bsky.actor.defs#viewerState'), py_type='app.bsky.actor.defs#profileViewBasic'), cid='bafyreifdfwogqmypniesjjwexpugki75ayrzegdizmh52hgw73zyffenem', indexed_at='2024-02-25T22:36:46.179Z', record=Record(created_at='2024-02-25T22:36:46.179Z', text='stop-tox', embed=None, entities=None, facets=None, labels=None, langs=['en'], reply=ReplyRef(parent=Main(cid='bafyreigpm2wngauduajaihuycdrumfrttjewnnk3x67xy6hklcfryu3wua', uri='at://did:plc:ygxp5kv5dzeaglhbysa7a5ph/app.bsky.feed.post/3kmbdqrp6gb2y', py_type='com.atproto.repo.strongRef'), root=Main(cid='bafyreigpm2wngauduajaihuycdrumfrttjewnnk3x67xy6hklcfryu3wua', uri='at://did:plc:ygxp5kv5dzeaglhbysa7a5ph/app.bsky.feed.post/3kmbdqrp6gb2y', py_type='com.atproto.repo.strongRef'), py_type='app.bsky.feed.post#replyRef'), tags=None, py_type='app.bsky.feed.post'), uri='at://did:plc:ygxp5kv5dzeaglhbysa7a5ph/app.bsky.feed.post/3kmblqtkadz2b', embed=None, labels=[], like_count=0, reply_count=0, repost_count=0, threadgate=None, viewer=ViewerState(like=None, reply_disabled=None, repost=None, py_type='app.bsky.feed.defs#viewerState'), py_type='app.bsky.feed.defs#postView'), parent=ThreadViewPost(post=PostView(author=ProfileViewBasic(did='did:plc:ygxp5kv5dzeaglhbysa7a5ph', handle='paritoshk.bsky.social', avatar='https://cdn.bsky.app/img/avatar/plain/did:plc:ygxp5kv5dzeaglhbysa7a5ph/bafkreifpgid6nusmgdrj7kbqyeu4admxtu42njoei5wvxz4oda36ty5lne@jpeg', display_name='Paritosh K', labels=[], viewer=ViewerState(blocked_by=False, blocking=None, blocking_by_list=None, followed_by=None, following=None, muted=False, muted_by_list=None, py_type='app.bsky.actor.defs#viewerState'), py_type='app.bsky.actor.defs#profileViewBasic'), cid='bafyreigpm2wngauduajaihuycdrumfrttjewnnk3x67xy6hklcfryu3wua', indexed_at='2024-02-25T20:13:34.362Z', record=Record(created_at='2024-02-25T20:13:34.362Z', text='@paradoxmn.bsky.social :gun today? gonna hack u now', embed=None, entities=None, facets=[Main(features=[Mention(did='did:plc:7zyq2rngtnmpdwcy6s6ibocc', py_type='app.bsky.richtext.facet#mention')], index=ByteSlice(byte_end=22, byte_start=0, py_type='app.bsky.richtext.facet#byteSlice'), py_type='app.bsky.richtext.facet')], labels=None, langs=['en'], reply=None, tags=None, py_type='app.bsky.feed.post'), uri='at://did:plc:ygxp5kv5dzeaglhbysa7a5ph/app.bsky.feed.post/3kmbdqrp6gb2y', embed=None, labels=[], like_count=0, reply_count=12, repost_count=0, threadgate=None, viewer=ViewerState(like=None, reply_disabled=None, repost=None, py_type='app.bsky.feed.defs#viewerState'), py_type='app.bsky.feed.defs#postView'), parent=None, replies=None, py_type='app.bsky.feed.defs#threadViewPost'), replies=[], py_type='app.bsky.feed.defs#threadViewPost')"
# Helper function to extract text from the thread object
def extract_text_from_thread(thread):
    """
    Extracts and compiles text from the provided thread object into a single string.

    :param thread: A structured object representing a thread with nested replies.
    :return: A string containing all text from the thread.
    """
    text_content = []
    # Extract text from the main post
    text_content.append(thread['post']['record']['text'])
    # Extract text from parent post if exists
    if thread.get('parent'):
        text_content.append(thread['parent']['post']['record']['text'])
    # Recursively extract text from replies if they exist
    if thread.get('replies'):
        for reply in thread['replies']:
            text_content.extend(extract_text_from_thread(reply))
    return " ".join(text_content)

# Function to detect toxic comments
def detect_toxic_comments(thread_info: str) -> list:
    detection_prompt = f"Identify any toxic comments from the following thread: {thread_info}"
    output = replicate.run(
        "google-deepmind/gemma-7b-it:2790a695e5dcae15506138cc4718d1106d0d475e6dca4b1d43f42414647993d5",
        input={
            "prompt": detection_prompt,
            "top_k": 40,
            "top_p": 0.9,
            "temperature": 0.5,
            "max_new_tokens": 100,
            "min_new_tokens": 50,
            "repetition_penalty": 1.2
        }
    )
    return output  # Adjust based on actual model output format

# Function to generate diffuser text
def generate_diffuser_text(thread_info: str, toxic_comments: list) -> str:
    toxic_summary = " ".join(toxic_comments)
    diffuser_prompt = f"With the context of this thread: {thread_info} and the following toxic comments: {toxic_summary}, generate a response that can help diffuse the situation."
    output = replicate.run(
        "google-deepmind/gemma-7b-it:2790a695e5dcae15506138cc4718d1106d0d475e6dca4b1d43f42414647993d5",
        input={
            "prompt": diffuser_prompt,
            "top_k": 40,
            "top_p": 0.9,
            "temperature": 0.5,
            "max_new_tokens": 100,
            "min_new_tokens": 50,
            "repetition_penalty": 1.2
        }
    )
    return output  # Adjust based on actual model output format

# Main script execution
if __name__ == "__main__":
    # Assuming `thread` variable is already defined as per your structure
    thread_text = extract_text_from_thread(thread)
    toxic_comments = detect_toxic_comments(thread_text)
    diffuser_text = generate_diffuser_text(thread_text, toxic_comments)
    print(diffuser_text)


TypeError: string indices must be integers, not 'str'